# MNIST classicifation

In [ ]:
# Imports
import torch 
import torchvision
from torchvision.transforms import ToTensor 
import torchvision.transforms as trnsfrms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
# Hyperparameters
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
# Check available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Load training data
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,)) # Using the mean and standard deviation of the dataset
                ]),
    download = True,            
), batch_size=batch_size_train, shuffle=True)

# Load test data
test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,)) # Using the mean and standard deviation of the dataset
                ]),
    download = True, 
), batch_size=batch_size_test, shuffle=True)

In [ ]:
# Visualize data
examples = enumerate(test_loader) # Iterates over all test data
batch_idx, (example_data, example_targets) = next(examples) # Take first batch
print(example_data.shape)
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])

In [ ]:
# Create model class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # Convolution, pooling, and relu activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) # Convolution, dropout, pooling, and relu activation
        x = x.view(-1, 320) # Reshape tensors
        x = F.relu(self.fc1(x)) # Fully connected layer and relu activation
        x = F.dropout(x, training=self.training) # Dropout
        x = self.fc2(x) # Fully connected layer
        return F.log_softmax(x) # Softmax layer

In [ ]:
# Define net object and optimizer
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
# Training history
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

# Train loop
def train(epoch):
  
  # Set network in training mode
  network.train()

  # Loop over all batches using the training loader
  for batch_idx, (data, target) in enumerate(train_loader):

    # Reset parameter gradients
    optimizer.zero_grad()

    # Forward pass
    output = network(data)

    # Compute loss
    loss = F.nll_loss(output, target)

    # Calculate new gradients
    loss.backward()

    # Backpropagate
    optimizer.step()

    # Logging
    if batch_idx % log_interval == 0:
      
      # Log
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

      # Save model
      torch.save(network.state_dict(), 'models/model.pth')
      torch.save(optimizer.state_dict(), 'models/optimizer.pth')


In [ ]:
# Test loop
def test():

  # Evaluate network on batch
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():

    # Loop over samples
    for data, target in test_loader:

      # Forward pass
      output = network(data)

      # Compute loss
      test_loss += F.nll_loss(output, target, size_average=False).item()

      # Network prediction
      pred = output.data.max(1, keepdim=True)[1]

      # Number of correct predictions
      correct += pred.eq(target.data.view_as(pred)).sum()

  # Accuracy
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

In [ ]:
# Test the network on randomly generated parameters
test()

# Loop over all epochs
for epoch in range(1, n_epochs + 1):
  
  # Train
  train(epoch)

  # Test
  test()

In [ ]:
# Plot training curve
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [ ]:
# Predict somee samples
with torch.no_grad():
  output = network(example_data)

# Plot
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])

### Continued training from checkpoints

In [ ]:
# Create new net and optimizer
continued_network = Net()
continued_optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

# Load pretrained network
network_state_dict = torch.load('models/model.pth')
continued_network.load_state_dict(network_state_dict)

# Load optimizer status
optimizer_state_dict = torch.load('models/optimizer.pth')
continued_optimizer.load_state_dict(optimizer_state_dict)

In [ ]:
# Retrain for 5 more epochs
for i in range(4,9):
  test_counter.append(i*len(train_loader.dataset))
  train(i)
  test()

In [ ]:
# Evaluate network
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')